### 从log文件输出信息  
`variable`  
`print`  
`variable`用于变量定义
`variable E0 equal etotal`   
`variable E_prefect equal ${E0}`  
由于`etotal`是系统的总能量值（特殊的词，LAMMPS中已定义为系统总能量，其它热力学信息特定词可详查`thermo_style`命令），会随着命令的执行而不断变化，所以这里连用两个`variable`命令来记录第一次优化后系统的能量值。  
`variable  N equal count(all)`  
`variable  N0 equal ${N}`  
同理，连用两个`variable`命令来记录第一次优化后的总原子数，`count（all）`表示计算总原子数目。  
而第二次（最后一次）优化后，由于`etotal`不再变化，则只需用一个`variable`命令记录。  

In [ ]:
%%writefile ex1.in
units metal
atom_style atomic 
boundary p p p 
neighbor 0.3 bin

lattice bcc 3.168
region box block  0 10 0 10 0 10
create_box 1 box
create_atoms 1 box

pair_style eam/alloy
pair_coeff * * W_zhou.eam.alloy W

thermo 10
dump 1 all custom 1 vacancy.xvz id type x y z

minimize 1.0e-12 1.0e-13 10000 10000

# 记录第一次优化后的能量值与总原子数目
variable E0 equal etotal
variable E_prefect equal ${E0}
variable N equal count(all)
variable N0 equal ${N}

region 2 sphere 5.0 5.0 5.0 0.1 side in
delete_atoms region 2

minimize 1.0e-12 1.0e-13 10000 10000
variable E_defect equal etotal
variable Ev equal (${E_defect}-((${N0}-1)/${N0})*${E_prefect})

print "ALL done"
print "Vacancy formation energy = ${Ev} ev"

In [ ]:
!"E:/LAMMPS/bin/lmp" < ex1.in